In [1]:
import pandas as pd
import numpy as np

In [2]:
data_folder = "../../Dataset/Movie/"

In [3]:
rating = pd.read_csv(f"{data_folder}ratings.csv").drop(["timestamp"], axis=1)
movie = pd.read_csv(f"{data_folder}movies.csv")
#tag = pd.read_csv(f"{data_folder}tags.csv").drop(["timestamp"], axis=1)
data = pd.merge(movie, rating, on="movieId")

In [4]:
data.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0


In [5]:
N = 1000
data_list = []
L = data.shape[0]

for i in range(1,N+1):
    debut = int((i-1)*L/N)
    fin = int(i*L/N)
    data_list.append(data.iloc[debut:fin, :])

In [6]:
pv_list = []

for d in data_list:
    pv_list.append(d.pivot_table(index="userId", columns="title", values="rating", dropna=False))

In [7]:
titles = data.title.unique()
dico = {title : [] for title in titles}

for pv in pv_list:
    for title in pv:
        dico[title].append(pv[title])

In [8]:
from IPython.display import clear_output

all_df = []
N = len(titles)

i = 0
for title in titles:
    tmp = pd.concat(dico[title], axis=0)
    if not (np.sum(tmp.isna(), axis=0) > tmp.shape[0]*0.3):
        all_df.append(tmp)
    else:
        del(tmp)

    if (i%300 == 0):
        clear_output(wait=True)
        print(f"{round(i*100/N, 1)}%")
    i += 1

99.7%


In [11]:
pd.concat(all_df, axis=1)

ValueError: cannot reindex from a duplicate axis

In [8]:
for pv in pv_list:
    ratings = []
    
    for title in titles:
        try:
            ratings.append(pv[title])
        except KeyError:
            pass

    

1000

In [62]:
cdata = data.copy()
N = 10**4
thresh = int(N*0.1)

ids = cdata["userId"].value_counts()[:N].index
cdata = cdata[cdata["userId"].isin(ids)]

titles = cdata["title"].value_counts()[:N].index
cdata = cdata[cdata["title"].isin(titles)]

pv = cdata.pivot_table(index="userId", columns="title", values="rating")
pv = pv.dropna(axis=1, thresh=thresh).dropna(axis=0, thresh=thresh)

pv

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
548,NaN,3.5,NaN,4.5,NaN,2.0,4.0,NaN,NaN,3.5,...,NaN,NaN,4.5,4.0,2.0,NaN,5.0,4.0,3.0,NaN
626,2.5,NaN,NaN,4.0,NaN,4.0,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0
847,2.5,NaN,NaN,3.5,NaN,NaN,3.5,4.0,4.5,3.0,...,3.5,NaN,4.0,NaN,3.5,3.0,NaN,NaN,NaN,2.5
997,NaN,4.0,4.0,3.0,2.5,3.0,3.0,4.0,3.5,3.5,...,4.0,NaN,4.0,3.5,3.0,4.0,NaN,3.0,3.5,NaN
1401,3.0,1.0,4.0,1.0,NaN,2.0,NaN,5.0,2.0,NaN,...,NaN,NaN,4.0,2.5,2.0,2.0,3.0,2.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161383,NaN,NaN,NaN,NaN,4.0,4.0,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
161586,NaN,NaN,NaN,NaN,0.5,0.5,2.5,3.5,NaN,NaN,...,NaN,NaN,2.0,2.5,2.5,NaN,1.0,1.0,1.5,1.0
162047,NaN,4.0,NaN,3.0,NaN,2.5,NaN,5.0,4.0,4.0,...,3.5,3.5,5.0,3.0,2.0,NaN,4.5,4.5,NaN,NaN


<p>Beaucoup de film sont très peu rempli, il faudrait faire un algorithme qui permet de faire un pivot table sur toutes les données.</p>
<p>Pour cela il faudrait découper le df par les colonnes, du style pleins de df en (:, 10) un truc comme ça.</p>
<p>puis on applique le dropna et à la fin on merged tous les df.</p>

In [ ]:
def add_only_new_elem(base_list:list, adding_list:list)->list:
    commun = [elem for elem in adding_list if elem in base_list]
    for elem in commun:
        adding_list.remove(elem)
    return adding_list

In [ ]:
genre = []
for g in data.genres.value_counts().index:
    genre = genre + add_only_new_elem(genre, g.split("|"))
genre.remove("IMAX")
genre.remove('(no genres listed)')